In [1]:
# Import the Earth Engine Python Package
import ee
from IPython.display import Image
import pprint
import pandas as pd

# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

In [2]:
# [lon, lat]

samplePointCoordinates = [[-69.782318,44.488825],
                          [-69.786357,44.520595],
                          [-69.854956,44.545618],
                          [-69.878186,44.560319],
                          [-69.761666,44.566191],
                          [-69.778516,44.604603],
                          [-69.779566,44.612670],
                          [-69.788011,44.614337]]



#samplePoints = ee.FeatureCollection(samplePointCoordinates)

samplePoints = []

for i in range(0,len(samplePointCoordinates)):
    point = ee.Geometry.Point(samplePointCoordinates[i])
    samplePoints.append(point)

allSamplePoints = ee.FeatureCollection(samplePoints)

# print(allSamplePoints)


In [3]:
# Generate a tile layer url
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

# Generate a feature collection layer url
def GetFeatureLayerUrl(ee_featurecollection_object):
    map_id = ee.FeatureCollection(ee_featurecollection_object).getMapId()
    featurecollection_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return featurecollection_url_template.format(**map_id)

In [13]:
import ipyleaflet 
belgrade = ipyleaflet.Map(center=(44.53,-69.81), zoom=11, layout={'height':'600px'})
dc = ipyleaflet.DrawControl()
belgrade.add_control(dc)
belgrade

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[44.53, -69.81], controls=(DrawControl(layer=FeatureGroup(), polygon={'shapeOptions': {}}, polyline={'shapeOptions': {}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']),), layout=Layout(height='600px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=11)

In [5]:
url = GetFeatureLayerUrl(allSamplePoints)

print(url)

belgrade.add_layer(ipyleaflet.TileLayer(url=url))

https://earthengine.googleapis.com/map/5f4c7d1e9df860708061c172644e61eb/{z}/{x}/{y}?token=40cc24665c1b7f2a56a99197aebb15fa


In [6]:
imageDates    = ["2012-09-12","2011-09-19","2011-08-18","2010-09-16","2010-08-31",
                 "2010-07-05","2010-06-19","2009-09-20","2009-09-04","2009-08-19",
                 "2009-07-11","2007-07-22","2006-10-07","2006-09-12","2006-08-11",
                 "2006-06-17","2005-08-17","2005-06-21","2004-08-14","2003-08-19"]
imageEndDates = ["2012-09-13","2011-09-20","2011-08-19","2010-09-17", "2010-09-01",
                 "2010-07-06","2010-06-20","2009-09-21", "2009-09-05","2009-08-20",
                 "2009-07-12","2007-07-23", "2006-10-08","2006-09-13","2006-08-12",
                 "2006-06-18", "2005-08-18","2005-06-22","2004-08-15","2003-08-20"]



In [7]:
landsatTOA = []   

for i in range(0,len(imageDates)):
    landsatImage = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").filterDate(imageDates[i],imageEndDates[i])
    landsatTOA.append(landsatImage)

In [8]:
'''
// 4. RETRIEVE REFLECTANCE VALUES FOR B1, B2, B3, B4
// (ORDER 1: date newest to oldest; 2: sample point number smallest to largest)

Image dates and on which corresponding sample point where in situ data were collected
(Note: same landsat image may be used for field data collected at the same location on different dates.)

2012-09-12:0,5,7
2011-09-19:6,7
2011-08-18:4,5,6,6,7
2010-09-16:5
2010-08-31:6,7
2010-07-05:2,3,3,3,7
2010-06-19:6
2009-09-20:7
2009-09-04:2,5,6,7
2009-08-19:2,3
2009-07-11:3,5,6,7
2007-07-22:6,7
2006-10-07:5,7
2006-09-12:5
2006-08-11:0,0,1,5,6,7
2006-06-17:6
2005-08-17:0,2
2005-06-21:5,7
2004-08-14:1,2
2003-08-19:2

'''

pointSelect = [[0,5,7],[6,7],[4,5,6,6,7],[5],[6,7],
                   [2,3,3,3,7],[6],[7],[2,5,6,7],[2,3],
                   [3,5,6,7],[6,7],[5,7],[5],[0,0,1,5,6,7],
                   [6],[0,2],[5,7],[1,2],[2]];

point = landsatTOA[0].getRegion(samplePoints[pointSelect[0][0]],1)

pp.pprint(point.get(0).getInfo())

pp.pprint(point.get(1).getInfo()[5])



['id',
 'longitude',
 'latitude',
 'time',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'sr_atmos_opacity',
 'sr_cloud_qa',
 'pixel_qa',
 'radsat_qa']
217


In [9]:
# MODIS 250

modis250 = []   

for i in range(0,len(imageDates)):
    modisImage = ee.ImageCollection("MODIS/006/MYD09GQ").filterDate(imageDates[i],imageEndDates[i])
    modis250.append(modisImage)


In [10]:
pointSelect = [[0,5,7],[6,7],[4,5,6,6,7],[5],[6,7],
                   [2,3,3,3,7],[6],[7],[2,5,6,7],[2,3],
                   [3,5,6,7],[6,7],[5,7],[5],[0,0,1,5,6,7],
                   [6],[0,2],[5,7],[1,2],[2]];

modis250point = modis250[0].getRegion(samplePoints[pointSelect[0][0]],1)

pp.pprint(modis250point.get(0).getInfo())

pp.pprint(modis250point.get(1).getInfo()[5])


['id',
 'longitude',
 'latitude',
 'time',
 'num_observations',
 'sur_refl_b01',
 'sur_refl_b02',
 'QC_250m',
 'obscov',
 'iobs_res',
 'orbit_pnt',
 'granule_pnt']
127


In [11]:
modis500 = []   

for i in range(0,len(imageDates)):
    modisImage = ee.ImageCollection("MODIS/006/MYD09GA").filterDate(imageDates[i],imageEndDates[i])
    modis500.append(modisImage)
    

In [12]:
pointSelect = [[0,5,7],[6,7],[4,5,6,6,7],[5],[6,7],
                   [2,3,3,3,7],[6],[7],[2,5,6,7],[2,3],
                   [3,5,6,7],[6,7],[5,7],[5],[0,0,1,5,6,7],
                   [6],[0,2],[5,7],[1,2],[2]];

modis500point = modis500[0].getRegion(samplePoints[pointSelect[0][0]],1)

pp.pprint(modis500point.get(0).getInfo())

pp.pprint(modis500point.get(1).getInfo()[15])

['id',
 'longitude',
 'latitude',
 'time',
 'num_observations_1km',
 'state_1km',
 'SensorZenith',
 'SensorAzimuth',
 'Range',
 'SolarZenith',
 'SolarAzimuth',
 'gflags',
 'orbit_pnt',
 'granule_pnt',
 'num_observations_500m',
 'sur_refl_b01',
 'sur_refl_b02',
 'sur_refl_b03',
 'sur_refl_b04',
 'sur_refl_b05',
 'sur_refl_b06',
 'sur_refl_b07',
 'QC_500m',
 'obscov_500m',
 'iobs_res',
 'q_scan']


SSLEOFError: EOF occurred in violation of protocol (_ssl.c:777)

In [ ]:
'''
b1 = []
# Use nested for loop to retrieve TOA of B1
for i in range(0,len(landsatTOA)):
    for j in range (0,len(pointSelect)):
        while j < i: 
            point = landsatTOA[i].getRegion(samplePoints[pointSelect[i][j]], 1)
            b1_refl = point.get(1).getInfo()[4]
            b1.append(b1_refl)

print(b1)

'''
